### Information 

**badAI**

This capstone project is built in the first place because after further discussion, our team gravitates more on the beauty industry because its a growing business with a lot of opportunities and room for growth. We are also interested in solving medically related problems within the scalp and help people identify it easier. So we are trying to tackle this side of the business, especially in the haircare and scalp world where information, knowledge, and extensive service is still very scarce.

For this project, we use the dataset from google images because there is no dataset that contains the images that we need. The dataset contains 6 scalp problems that are alopecia areata, dandruff, folliculitis, psoriasis, seborrheic dermatitis, and tinea capitis and 1 dataset for healthy scalp.




### Environment Setup

#### Import Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_hub as hub
import numpy as np

#### Mount Drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Capstone-Dataset-C22-P222')
root_path = '/content/gdrive/My Drive/Capstone-Dataset-C22-P222/'

#### Assign Directories

In [ ]:
train_dir = os.path.join(root_path, 'train')
validation_dir = os.path.join(root_path, 'validation')

#### Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   rotation_range=360,
                                   shear_range=0.4,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

validation_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    shuffle=True,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    color_mode='rgb',
                                                    target_size=(150, 150))     

validation_generator =  validation_datagen.flow_from_directory(validation_dir,
                                                               shuffle=True,
                                                               batch_size=20,
                                                               color_mode='rgb',
                                                               class_mode  = 'categorical',
                                                               target_size = (150, 150))

#### Assign Transfer Learning (Inception V3)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

#### Assign Pre-Trained Model

In [ ]:
def create_pre_trained_model(local_weights_file):
  pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                  include_top = False, 
                                  weights = None) 

  pre_trained_model.load_weights(local_weights_file)

  for layer in pre_trained_model.layers:
    layer.trainable = False

  return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)

In [ ]:
def output_of_last_layer(pre_trained_model):
  last_desired_layer = pre_trained_model.get_layer('mixed7')
  print('last layer output shape: ', last_desired_layer.output_shape)
  last_output = last_desired_layer.output
  print('last layer output: ', last_output)

  return last_output

In [ ]:
last_output = output_of_last_layer(pre_trained_model)

#### Create Final Model

In [ ]:
def create_final_model(pre_trained_model, last_output):
  x = layers.Flatten()(last_output)

  x = layers.Dense(1024, activation='relu')(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(7, activation='softmax')(x)       

  model = Model(inputs=pre_trained_model.input, outputs=x)

  model.compile(optimizer = 'adam', 
                loss = 'categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [ ]:
model = create_final_model(pre_trained_model, last_output)

total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

#### Run The Model

In [ ]:
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 50,
                    verbose = 2,
                    steps_per_epoch = train_generator.samples // train_generator.batch_size,
                    validation_steps = validation_generator.samples // validation_generator.batch_size)

#### Accuracy and Loss Graph

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

#### Testing Image



In [ ]:
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  root_path = '/content/gdrive/My Drive/Capstone-Dataset/'
  path = root_path + fn
  img = image.load_img(path, target_size=(150, 150))

  # + " (2)"
  
  x=image.img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][0]:
    print(fn + " is alopecia_areata")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][1]:
    print(fn + " is dandruff")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][2]:
    print(fn + " is folliculitis")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][3]:
    print(fn + " is healthy_scalp")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][4]:
    print(fn + " is psoriasis")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][5]:
    print(fn + " is seborrheic_dermatitis")
  elif max([classes[0][0], classes[0][1], classes[0][2], classes[0][3], classes[0][4], classes[0][5], classes[0][6]]) == classes[0][6]:
    print(fn + " is tinea_capitis")

#### Save the Model into .h5

In [ ]:
model.save('Categorical_Adam.h5')